<a href="https://colab.research.google.com/github/PraveenSH/adversarial-generator-retriever/blob/main/generator_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

Load Generator

In [ ]:
MODEL_ID = "google-t5/t5-base"

gen_tokenizer = T5Tokenizer.from_pretrained(MODEL_ID)
gen_model = T5ForConditionalGeneration.from_pretrained(MODEL_ID)

checkpoint = torch.load("/content/drive/MyDrive/Generator_Retriever/Generator_Pretrained/model.pt_1_2000")
gen_model.load_state_dict(checkpoint['model_state_dict'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

<All keys matched successfully>

In [ ]:
'''
gen_model.eval()

prod_title = "coelia ombre gray area rug"
prod_desc = "woven on state of the art machines for ultimate durability and luxury , this rug is an elegant and sophisticated solution for your design needs . the rug is        crafted with deliciously soft viscose fibers on a soft cotton weft for unbeatable comfort underfoot . the rug features a high sheen , plush silken touch with intricate layers of color which shift beautif       ully depending on the angle from which you view the rug . the subtle distressing effect adds on trend , vintage appeal . shedding of loose fibers is normal with this construction . this will diminish with regular vacuuming and wear"
input_sequence = "Title: " + prod_title + " - Description: " + prod_desc

input_ids = gen_tokenizer([input_sequence, input_sequence], return_tensors="pt").input_ids
print(input_ids.size())
print(f'Input: {input_sequence}')

nsent = 4
with torch.no_grad():
    for i in range(nsent):
        output = gen_model.generate(input_ids, max_length=35, num_beams=1, do_sample=True, repetition_penalty=1.8)
        for i in range(len(output)):
          target_sequence = gen_tokenizer.decode(output[i], skip_special_tokens=True)
          print(f'Target: {target_sequence}')
'''

'\ngen_model.eval()\n\nprod_title = "coelia ombre gray area rug"\nprod_desc = "woven on state of the art machines for ultimate durability and luxury , this rug is an elegant and sophisticated solution for your design needs . the rug is        crafted with deliciously soft viscose fibers on a soft cotton weft for unbeatable comfort underfoot . the rug features a high sheen , plush silken touch with intricate layers of color which shift beautif       ully depending on the angle from which you view the rug . the subtle distressing effect adds on trend , vintage appeal . shedding of loose fibers is normal with this construction . this will diminish with regular vacuuming and wear"\ninput_sequence = "Title: " + prod_title + " - Description: " + prod_desc\n\ninput_ids = gen_tokenizer([input_sequence, input_sequence], return_tensors="pt").input_ids\nprint(input_ids.size())\nprint(f\'Input: {input_sequence}\')\n\nnsent = 4\nwith torch.no_grad():\n    for i in range(nsent):\n        output = ge

Load Retriever

In [ ]:
MODEL_ID = "google-bert/bert-large-uncased"
ret_tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
ret_model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)

checkpoint_path = "/content/drive/MyDrive/Generator_Retriever/Retriever_Pretrained/model.pt_1_1000"
checkpoint = torch.load(checkpoint_path)
ret_model.load_state_dict(checkpoint['model_state_dict'])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
'''
ret_model.eval()
import torch.nn.functional as F
query_text = "horizontal coastal look chair"
prod_text = "laplante beveled accent mirror  showcasing a casual coastal look , this square mirror is finished in a driftwood tone with a distressed whitewash . mirror has a generous 1 1/4 '' bevel and may be hung horizontal or vertical"
tok_output = ret_tokenizer(query_text, prod_text, padding='max_length',
                      max_length=160, truncation=True, return_tensors='pt',
                      return_attention_mask=True)

tok_output = ret_tokenizer([query_text, "query_text"], [prod_text, "prod_text"], padding='max_length',
                      max_length=160, truncation=True, return_tensors='pt',
                      return_attention_mask=True)
print(tok_output)

input_ids = tok_output.input_ids
attention_masks = tok_output.attention_mask
token_type_ids = tok_output.token_type_ids

output = ret_model(input_ids, attention_mask = attention_masks,
                   token_type_ids = token_type_ids, labels = torch.tensor([1, 0]))
print(output.loss)
probs = F.softmax(output.logits, dim=-1)
probs[:,1]
'''

'\nret_model.eval()\nimport torch.nn.functional as F\nquery_text = "horizontal coastal look chair"\nprod_text = "laplante beveled accent mirror  showcasing a casual coastal look , this square mirror is finished in a driftwood tone with a distressed whitewash . mirror has a generous 1 1/4 \'\' bevel and may be hung horizontal or vertical"\ntok_output = ret_tokenizer(query_text, prod_text, padding=\'max_length\',\n                      max_length=160, truncation=True, return_tensors=\'pt\',\n                      return_attention_mask=True)\n\ntok_output = ret_tokenizer([query_text, "query_text"], [prod_text, "prod_text"], padding=\'max_length\',\n                      max_length=160, truncation=True, return_tensors=\'pt\',\n                      return_attention_mask=True)\nprint(tok_output)\n\ninput_ids = tok_output.input_ids\nattention_masks = tok_output.attention_mask\ntoken_type_ids = tok_output.token_type_ids\n\noutput = ret_model(input_ids, attention_mask = attention_masks,\n     

Load a relevance model

In [ ]:
!pip install sentence_transformers==2.7.0
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

from sentence_transformers import SentenceTransformer
rel_model = SentenceTransformer('BlueAvenir/sustainability_relevance_class_model')
#rel_model.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
'''
from sklearn.metrics.pairwise import cosine_similarity

def get_sim(q_embeddings, p_embeddings):
  return (cosine_similarity(q_embeddings.reshape(1, -1), p_embeddings.reshape(1, -1))[0][0] + 1) / 2

  sims = []
  all_sims = cosine_similarity(q_embeddings, p_embeddings)
  for i in range(len(q_embeddings)):
    sims.append(all_sims[i][i])
  return sims

queries = "driftwood mirror"
prods = "laplante beveled accent mirror  showcasing a casual coastal look , this square mirror is finished in a driftwood tone with a distressed whitewash . mirror has a generous 1 1/4 '' bevel and may be hung horizontal or vertical"
q_embeddings = rel_model.encode(queries)
p_embeddings = rel_model.encode(prods)

print(get_sim(p_embeddings, q_embeddings))
'''

'\nfrom sklearn.metrics.pairwise import cosine_similarity\n\ndef get_sim(q_embeddings, p_embeddings):\n  return (cosine_similarity(q_embeddings.reshape(1, -1), p_embeddings.reshape(1, -1))[0][0] + 1) / 2\n\n  sims = []\n  all_sims = cosine_similarity(q_embeddings, p_embeddings)\n  for i in range(len(q_embeddings)):\n    sims.append(all_sims[i][i])\n  return sims\n\nqueries = "driftwood mirror"\nprods = "laplante beveled accent mirror  showcasing a casual coastal look , this square mirror is finished in a driftwood tone with a distressed whitewash . mirror has a generous 1 1/4 \'\' bevel and may be hung horizontal or vertical"\nq_embeddings = rel_model.encode(queries)\np_embeddings = rel_model.encode(prods)\n\nprint(get_sim(p_embeddings, q_embeddings))\n'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_relevance_reward(rel_model, query, product):

  q_emb = rel_model.encode(query)
  p_emb = rel_model.encode(product)

  sim = cosine_similarity(q_emb.reshape(1, -1), p_emb.reshape(1, -1))[0][0]
  return sim
  #return (sim + 1) / 2

In [ ]:
import torch.nn.functional as F

def train_generator_retriever(dataloader, generator, retriever, relevance_model,
                              gen_optimizer, ret_optimizer, gen_scheduler,
                              gen_tokenizer, ret_tokenizer,
                              ret_scheduler, device, epoch_num):
  ret_total_loss = 0
  gen_total_loss = 0
  steps = 0

  for batch in dataloader:
   for it in range(2):
    steps += 1
    generator.eval()
    #Generator generates k queries for each sample in the batch. New batch of (batch*k)
    prod_input_ids = batch["input_ids"]
    gen_query_ids = generator.generate(prod_input_ids, max_length=35, num_beams=1,
                                do_sample=True, top_k=0, repetition_penalty=1.8)
    gen_queries = []
    prod_descs = []
    neg_queries = batch["neg_queries"]

    for i in range(len(gen_query_ids)):
      gen_sequence = gen_tokenizer.decode(gen_query_ids[i], skip_special_tokens=True)
      prod_sequence = gen_tokenizer.decode(prod_input_ids[i], skip_special_tokens=True)

      gen_queries.append(gen_sequence)
      prod_descs.append(prod_sequence)

    rewards = []
    #Collect losses of retriever/ reward for generator
    for i in range(len(gen_queries)):

        retriever.eval()
        query_text = gen_queries[i]
        prod_text = prod_descs[i]
        neg_query = neg_queries[i]
        tok_output = ret_tokenizer(query_text, prod_text, padding='max_length',
                        max_length=160, truncation=True, return_tensors='pt',
                        return_attention_mask=True)


        input_ids = tok_output.input_ids
        attention_masks = tok_output.attention_mask
        token_type_ids = tok_output.token_type_ids

        output = retriever(input_ids, attention_mask = attention_masks,
                          token_type_ids = token_type_ids)
        #reward for generator - score of irrelevant class -> maximize
        ret_reward = (F.softmax(output.logits, dim=-1)[0][0] - 0.5) * 2.0



        #Inner loop train retriever with above.
        retriever.train()
        output_pos = retriever(input_ids, attention_mask = attention_masks,
                           token_type_ids = token_type_ids,
                           labels = torch.tensor([1]))

        tok_output = ret_tokenizer(neg_query, prod_text, padding='max_length',
                        max_length=160, truncation=True, return_tensors='pt',
                        return_attention_mask=True)
        input_ids = tok_output.input_ids
        attention_masks = tok_output.attention_mask
        token_type_ids = tok_output.token_type_ids

        output_neg = retriever(input_ids, attention_mask = attention_masks,
                           token_type_ids = token_type_ids,
                           labels = torch.tensor([0]))

        ret_loss = output_pos.loss + output_neg.loss
        ret_total_loss += ret_loss

        ret_optimizer.zero_grad()
        ret_loss.backward()
        ret_optimizer.step()
        ret_scheduler.step()

        rel_reward = get_relevance_reward(relevance_model, query_text, prod_text)

        alpha = 0.4
        final_reward = alpha * rel_reward + (1 - alpha) * ret_reward
        #if i < 5:
        # print(it, query_text, ret_reward, rel_reward, prod_text)

        rewards.append(final_reward)

    #Update Generator with losses and relevance as reward. -> policy gradient loss
    generator.train()
    logits = generator(prod_input_ids, labels = gen_query_ids).logits
    log_probs = torch.log(F.softmax(logits, dim=-1))
    gathered_log_probs = torch.gather(log_probs, 2, gen_query_ids.unsqueeze(-1)).squeeze()
    reward_tensor = torch.tensor(rewards).view(-1, 1)
    gen_loss = -torch.mul(gathered_log_probs, reward_tensor).sum(dim=-1).mean()
    #print(gen_loss)
    gen_total_loss += gen_loss

    gen_optimizer.zero_grad()
    gen_loss.backward()
    gen_optimizer.step()
    gen_scheduler.step()

    if ((steps%log_steps) == 0):
            print(steps, ret_total_loss/steps, gen_total_loss/steps)

    if ((steps%save_steps) == 0):
          torch.save({'model_state_dict': gen_model.state_dict(), 'optimizer_state_dict': gen_optimizer.state_dict()}, gen_save_path+"_"+str(epoch_num)+"_"+str(steps))
          torch.save({'model_state_dict': ret_model.state_dict(), 'optimizer_state_dict': ret_optimizer.state_dict()}, ret_save_path+"_"+str(epoch_num)+"_"+str(steps))


In [ ]:
def load_dataset(file_path):
    df = pd.read_csv(file_path, sep='\t')
    results = df[['query', 'product_name', 'product_description']].fillna("").values.tolist()
    return results

class ProductDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        query, product_name, product_description = self.data[index]
        input_text = "Title: " + product_name + " - Description: " + product_description

        input_ids = self.tokenizer(input_text, padding='max_length', max_length=max_prod_len, truncation=True, return_tensors='pt').input_ids
        return {'input_ids': input_ids[0], 'neg_queries': query}

In [ ]:
gen_save_path = "/content/drive/MyDrive/Generator_Retriever/Generator/model.pt"
ret_save_path = "/content/drive/MyDrive/Generator_Retriever/Retriever/model.pt"
data_path = "/content/drive/MyDrive/WANDS/non_pretrain_neg.csv"
max_query_len = 35
max_prod_len = 128
log_steps = 10
save_steps = 30
batch_size = 8
num_epochs = 4
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

data = load_dataset(data_path)
import random
random.shuffle(data)

dataset = ProductDataset(data[:5000], gen_tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

gen_optimizer = AdamW(gen_model.parameters(), lr=1e-5)
gen_scheduler = get_linear_schedule_with_warmup(gen_optimizer, num_warmup_steps=200,
                                                num_training_steps = num_epochs * (len(dataset) // batch_size))

ret_optimizer = AdamW(ret_model.parameters(), lr=2e-5)
ret_scheduler = get_linear_schedule_with_warmup(ret_optimizer, num_warmup_steps=200,
                                                num_training_steps = num_epochs * (len(dataset) // batch_size))

for epoch_num in range(num_epochs):
    train_generator_retriever(dataloader, gen_model, ret_model, rel_model,
                                  gen_optimizer, ret_optimizer, gen_scheduler,
                                  gen_tokenizer, ret_tokenizer,
                                  ret_scheduler, device, epoch_num)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


10 tensor(3.9572, grad_fn=<DivBackward0>) tensor(-17.3024, grad_fn=<DivBackward0>)
20 tensor(3.9348, grad_fn=<DivBackward0>) tensor(-17.9963, grad_fn=<DivBackward0>)
30 tensor(4.0126, grad_fn=<DivBackward0>) tensor(-15.6529, grad_fn=<DivBackward0>)
40 tensor(3.8984, grad_fn=<DivBackward0>) tensor(-16.1881, grad_fn=<DivBackward0>)
50 tensor(4.5095, grad_fn=<DivBackward0>) tensor(-14.5680, grad_fn=<DivBackward0>)
60 tensor(4.7724, grad_fn=<DivBackward0>) tensor(-13.6612, grad_fn=<DivBackward0>)
70 tensor(4.8400, grad_fn=<DivBackward0>) tensor(-13.1678, grad_fn=<DivBackward0>)
80 tensor(4.8534, grad_fn=<DivBackward0>) tensor(-13.2707, grad_fn=<DivBackward0>)
